In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext rpy2.ipython

In [6]:
import numpy as np
import pandas as pd

from kaggle.house_prices import helpers
from kaggle.house_prices import missing
from kaggle.house_prices import outliers

combined_dataset = helpers.load_data()
combined_dataset = missing.fix_all(combined_dataset)
combined_dataset = outliers.remove_outliers(combined_dataset)
combined_dataset['price_log'] = np.log(combined_dataset['SalePrice'])
combined_dataset = combined_dataset.drop(['SalePrice', 'Id'], axis=1)
combined_dataset.shape

(2917, 81)

In [9]:
#
# Global quantiles
#
from kaggle.house_prices.quantile_rating import calc_quantiles

calc_quantiles(
    sample=combined_dataset['price_log'],
    probs=[0.25, 0.5, 0.75]
)

0.25    11.774713
0.50    12.001505
0.75    12.273731
Name: price_log, dtype: float64

In [14]:
#
# Quantile ratings calculation
#
from kaggle.house_prices.quantile_rating import calc_ratings

ratings1 = calc_ratings(
    df=combined_dataset, 
    target_var='price_log', 
    rating_quantiles=calc_quantiles(
        sample=combined_dataset['price_log'],
        probs=[0.25, 0.5, 0.75]
    ), 
    categ_vars=helpers.get_character_colnames(combined_dataset)
)

In [17]:
ratings1.sort_values(['var', 'rating']).head(9)

,var,value,rating
0,Alley,Grvl,1.500000
2,Alley,_none_,2.531822
1,Alley,Pave,2.536585
4,BldgType,2fmCon,1.580645
5,BldgType,Duplex,1.692308
6,BldgType,Twnhs,1.883721
3,BldgType,1Fam,2.564039
7,BldgType,TwnhsE,2.622807
10,BsmtCond,Po,1.000000


In [18]:
ratings1.sort_values(['var', 'rating']).tail(9)

,var,value,rating
288,SaleType,WD,2.421468
281,SaleType,CWD,3.000000
286,SaleType,New,3.583333
282,SaleType,Con,4.000000
289,Street,Grvl,1.833333
290,Street,Pave,2.499311
292,Utilities,NoSeWa,2.000000
291,Utilities,AllPub,2.496911
293,NaN,NaN,2.500000
